## Importação das bibliotecas

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Definição do modelo CNN

In [2]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x



# Carregamento e transformação dos conjunto de dados CIFAR-10 

In [3]:

transform = transforms.Compose([
    transforms.ToTensor(),∞
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



Files already downloaded and verified
Files already downloaded and verified


# Definição da função de perda e otimizador 

In [4]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Treinamento da rede

In [7]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.189
[1,  4000] loss: 1.186
[1,  6000] loss: 1.184
[1,  8000] loss: 1.177
[1, 10000] loss: 1.161
[1, 12000] loss: 1.157
[2,  2000] loss: 1.095
[2,  4000] loss: 1.106
[2,  6000] loss: 1.082
[2,  8000] loss: 1.079
[2, 10000] loss: 1.086
[2, 12000] loss: 1.090
[3,  2000] loss: 1.013
[3,  4000] loss: 1.022
[3,  6000] loss: 1.001
[3,  8000] loss: 1.029
[3, 10000] loss: 1.036
[3, 12000] loss: 1.031
[4,  2000] loss: 0.931
[4,  4000] loss: 0.945
[4,  6000] loss: 0.966
[4,  8000] loss: 0.976
[4, 10000] loss: 0.995
[4, 12000] loss: 0.992
[5,  2000] loss: 0.894
[5,  4000] loss: 0.925
[5,  6000] loss: 0.898
[5,  8000] loss: 0.930
[5, 10000] loss: 0.921
[5, 12000] loss: 0.928
Finished Training


# Teste da rede nos dados de teste


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 63 %
